## Image pulling - Landat 7 / Landast 8 - primary source GEE API
##### This code calls API and pull images to local computer and store by the day and create a image catalogue out, input: location of image, output: raw image and metadata collection
##### Author: Yahampath Marambe, full readme at github A-Marambe

In [1]:
# imports for data handling 
import numpy as np
import pandas as pd

# imports for rasters
from osgeo import gdal
from osgeo import ogr
from osgeo import osr

# imports for vectors
import shapely
import fiona
import geopandas as gpd
import json

# import data vizualization
from matplotlib import pyplot as plt
import folium
import IPython.display as disp

# system imports
import os
import glob
import shutil
from datetime import datetime

# image sourcing
import ee
import requests
# import geetools as gt : cloud mask oprions
# intializing authenticated API client
ee.Initialize()
%matplotlib inline

## Image collection functions
###### create image collection fucnctions to call collection filter for date, cloulds, AOI, and band combinations if needed

In [2]:
# laoding images
def call_filtered_images(collection, time_range, area):
    """ create image collection from GEE API
    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest, converted geojson

    Returns:
        image collection
     """
    collection = ee.ImageCollection(collection)
    ## Filter by time and location
    temporalFiltered = collection.filterDate(time_range[0], time_range[1])
    spatialFiltered = temporalFiltered.filterBounds(area)
    cloudFiltered = spatialFiltered.sort('CLOUD_COVER')
    imagecol = cloudFiltered.first()
    #image = ee.Image(imagecol)
    return imagecol

# display images
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    """
    display images from API on foluim open street maps
    parameters: image, visual parameters (max, min, gamma)m layer name
    output: image on the OSM
    """
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

# cloud mask functions for ls8
def cloudMaskLS8(image):
    """
    input ls8 sr image
    output cloud and shadow masked image
    """
    # select the quality band
    qa = image.select('QA_PIXEL') ##substitiu a band FMASK
    # build masks
    cloud1 = qa.bitwiseAnd(1<<5).eq(0)
    cloud2 = qa.bitwiseAnd(1<<2).eq(0)
    cloud3 = qa.bitwiseAnd(1<<3).eq(0)
    cloud4 = qa.bitwiseAnd(1<<4).eq(0)
    #  apply masks
    return image.updateMask(cloud1).updateMask(cloud2).updateMask(cloud3).updateMask(cloud4)

# convert image to geotiff
def get_url(name, image, scale, region):
    """It will open and download automatically a zip folder containing Geotiff data of 'image'.
    If additional parameters are needed, see also:

    Parameters:
        name (str): name of the created folder
        image (ee.image.Image): image to export
        scale (int): resolution of export in meters (e.g: 30 for Landsat)
        region (list): region of interest

    Returns:
        path (str)
     """
    path = image.getDownloadURL({
        'name':(name),
        'scale': scale,
        'region':(region)
        })

    webbrowser.open_new_tab(path)
    return path

In [3]:
# user input
##########################
start_date = '2018-01-01'
end_date = '2018-06-30'  
# sensor, image collection
landsat8 = "LANDSAT/LC08/C02/T1_L2"
# requesting bands
bands = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','ST_B10','QA_PIXEL']

In [4]:
# path list of landslides
box_list = glob.glob('./landslide_box/*.geojson')
# image list for display
disp_list = []
name_list = []
# create date range
intervals = pd.date_range(start=start_date, end=end_date, freq='5D')

# taking 5 day ranges and loading images
for i in range (0,len(intervals)):
    ii = i+1
    if i < len(intervals)-1:
        st_day = intervals[i].strftime('%Y-%m-%d')
        en_day = intervals[ii].strftime('%Y-%m-%d')
        #print('start {day1} end {day2}'.format(day1=st_day,day2=en_day))
    # laoding images
    for item in box_list:
        print(item[16:-8])
        # open json file
        js = open(item)
        # load json
        geojsn = json.load(js)
        # print(geojsn)
        # extract coordinates
        coords = geojsn['features'][0]['geometry']['coordinates']
        #print(coords)
        # cast to ee geometry
        aoi = ee.Geometry.MultiPolygon(coords)
        # time range
        time_range = ([st_day, en_day])
        # call image from function
        ls8 = call_filtered_images(landsat8, time_range, aoi)
        # print sensing day
        try:
            lsdict = ls8.date().getInfo()
            # extract sensing day
            lsinfo = float(str(list(lsdict.values())[1])[:10])
            # convert from timestamp
            sensing_day = (datetime.fromtimestamp(lsinfo)).strftime("%Y-%m-%d")
            print('image was captured on {val}'.format(val=sensing_day))
            #clip image
            cliped_image = ls8.clip(aoi)
            #cloud mask ls8: note: sentinel is different
            cloud_masked = cloudMaskLS8(cliped_image).select(bands)
            # get downloadeble url
            url = cloud_masked.getDownloadURL({'scale':30, 'filePerBand':False, 'region':aoi, 'maxPixels': 1e13})
            print(url)
            print('start downloading {} image'.format(item[16:-8]))
            # download the url: just clicking on the link do the same job
            r = requests.get(url)
            with open('./ls8_images/'+item[16:-8]+'-'+sensing_day+'.zip', 'wb') as f:
                f.write(r.content)
            print(r.status_code)
            print('download successful')
            disp_list.append(cloud_masked)
            name_list.append(item[16:-8]+'-'+sensing_day)
        except:
            print('no image data available for {d1} and {d2} time range.'.format(d1=st_day, d2=en_day))

aranayake
1
no image data available for 2018-01-01 and 2018-01-06 time range.
kegall
1
no image data available for 2018-01-01 and 2018-01-06 time range.
kandyBox
1
no image data available for 2018-01-01 and 2018-01-06 time range.
aranayake
1
no image data available for 2018-01-06 and 2018-01-11 time range.
kegall
1
no image data available for 2018-01-06 and 2018-01-11 time range.
kandyBox
1
no image data available for 2018-01-06 and 2018-01-11 time range.
aranayake
1
no image data available for 2018-01-11 and 2018-01-16 time range.
kegall
1
no image data available for 2018-01-11 and 2018-01-16 time range.
kandyBox
1
no image data available for 2018-01-11 and 2018-01-16 time range.
aranayake
1
2
image was captured on 2018-01-16
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/e6caae68caa98ef682ea2089ea757010-751b75664229754dc4a4651e0f82b67e:getPixels
start downloading aranayake image
200
download successful
kegall
1
2
image was captured on 2018-01-16
htt

In [9]:
name_list

['aranayake-2018-01-16',
 'kegall-2018-01-16',
 'kandyBox-2018-01-16',
 'aranayake-2018-02-01',
 'kegall-2018-02-01',
 'kandyBox-2018-02-01',
 'aranayake-2018-02-17',
 'kegall-2018-02-17',
 'kandyBox-2018-02-17',
 'aranayake-2018-03-05',
 'kegall-2018-03-05',
 'kandyBox-2018-03-05',
 'aranayake-2018-03-21',
 'kegall-2018-03-21',
 'kandyBox-2018-03-21',
 'aranayake-2018-04-06',
 'kegall-2018-04-06',
 'kandyBox-2018-04-06',
 'aranayake-2018-04-22',
 'kegall-2018-04-22',
 'kandyBox-2018-04-22',
 'aranayake-2018-05-08',
 'kegall-2018-05-08',
 'kandyBox-2018-05-08',
 'aranayake-2018-05-24',
 'kegall-2018-05-24',
 'kandyBox-2018-05-24',
 'aranayake-2018-06-09',
 'kegall-2018-06-09',
 'kandyBox-2018-06-09',
 'aranayake-2018-06-25',
 'kegall-2018-06-25',
 'kandyBox-2018-06-25',
 'aranayake-2018-06-25',
 'kegall-2018-06-25',
 'kandyBox-2018-06-25']

In [11]:
# center location
location = aoi.centroid().coordinates().getInfo()[::-1]
# create a map object
m = folium.Map(location=location, zoom_start=12)


for item, name in zip(disp_list, name_list):
    swir = ee.Image.rgb(item.select('SR_B7'),
                        item.select('SR_B5'),
                        item.select('SR_B4'))
    m.add_ee_layer(swir, {'min': [0, 0, 0], 'max': [30000, 30000, 30000], 'gamma': [0.9, 0.9, 0.9]}, name)
    
    
m.add_child(folium.LayerControl())
display(m)


Display images as pngs
1. create first main catalogue

In [46]:
# switch to the image folder
temp_av = os.listdir('/home/anux/Desktop/geoSpYa/phenols/imagepull/ls8_images')
# list of available zip files with removed .zip extension
aval_list = [item[:-4] for item in temp_av]
# remove unwated 
aval_list.remove('.ipynb_checkpo')
#print(aval_list)
print('*****************')
# take image day
day_list = [item[-10:] for item in aval_list]
print(day_list)
# area name
area_list = [item[:-11] for item in aval_list]
print(area_list)
# make a cat df
cat_df = pd.DataFrame()
cat_df['date'] = day_list
cat_df['areaName'] = area_list
cat_df

*****************
['2018-02-17', '2018-05-24', '2018-02-01', '2018-05-24', '2018-02-01', '2018-06-09', '2018-05-08', '2018-02-17', '2018-01-16', '2018-05-08', '2018-04-06', '2018-06-25', '2018-04-06', '2018-01-16', '2018-01-16', '2018-04-22', '2018-03-21', '2018-03-05', '2018-05-08', '2018-06-09', '2018-03-05', '2018-06-09', '2018-04-22', '2018-03-21', '2018-06-25', '2018-04-22', '2018-02-01', '2018-03-21', '2018-06-25', '2018-05-24', '2018-02-17', '2018-04-06', '2018-03-05']
['aranayake', 'aranayake', 'aranayake', 'kegall', 'kandyBox', 'kandyBox', 'kandyBox', 'kegall', 'kandyBox', 'aranayake', 'aranayake', 'aranayake', 'kandyBox', 'kegall', 'aranayake', 'kegall', 'kandyBox', 'kegall', 'kegall', 'aranayake', 'aranayake', 'kegall', 'aranayake', 'aranayake', 'kandyBox', 'kandyBox', 'kegall', 'kegall', 'kegall', 'kandyBox', 'kandyBox', 'kegall', 'kandyBox']


,date,areaName
0,2018-02-17,aranayake
1,2018-05-24,aranayake
2,2018-02-01,aranayake
3,2018-05-24,kegall
4,2018-02-01,kandyBox
5,2018-06-09,kandyBox
6,2018-05-08,kandyBox
7,2018-02-17,kegall
8,2018-01-16,kandyBox
9,2018-05-08,aranayake


In [65]:
from zipfile import ZipFile
import glob

In [66]:
os.getcwd()

'/home/anux/Desktop/geoSpYa/phenols/imagepull/ls8_images'

In [123]:
# unzip folders
# list zip files
zip_files = zip_list = glob.glob('*')

for file in zip_files:
    if file == 'tiff_images':
        continue
    else:
        print('file name is ',file)
        area_name = file[:-15]
        print('area is ', area_name)
        zf = ZipFile(file)
        dirname = file[:-4]+'_down'
        print(dirname)
        os.mkdir('./images/'+dirname)
        zf.extractall('./images/'+dirname)
        #os.rename('./images/'+dirname, )
        print('********************************')
        
        # renametiff files in the folder

file name is  aranayake-2018-02-17.zip
area is  aranayake
aranayake-2018-02-17_down
********************************
file name is  aranayake-2018-05-24.zip
area is  aranayake
aranayake-2018-05-24_down
********************************
file name is  aranayake-2018-02-01.zip
area is  aranayake
aranayake-2018-02-01_down
********************************
file name is  kegall-2018-05-24.zip
area is  kegall
kegall-2018-05-24_down
********************************
file name is  kandyBox-2018-02-01.zip
area is  kandyBox
kandyBox-2018-02-01_down
********************************
file name is  kandyBox-2018-06-09.zip
area is  kandyBox
kandyBox-2018-06-09_down
********************************
file name is  kandyBox-2018-05-08.zip
area is  kandyBox
kandyBox-2018-05-08_down
********************************
file name is  kegall-2018-02-17.zip
area is  kegall
kegall-2018-02-17_down
********************************
file name is  kandyBox-2018-01-16.zip
area is  kandyBox
kandyBox-2018-01-16_down
**********

IsADirectoryError: [Errno 21] Is a directory: 'images'

In [61]:
# unzip and renaming files
# unzip
zf = ZipFile('aranayake-2018-02-17.zip')
# extract
zf.extractall('tiff_images')
# rename all files
os.rename('./tiff_images/LC08_141055_20180217.tif', './tiff_images/aranayake-141055-2018-02-17.tif')